### 코렙에서 한글 사용

In [ ]:
!apt -qq -y install fonts-nanum
 
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
 
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
fm._rebuild()
 
# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'
 
# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic') 

### 구글드라이브 연동

In [6]:
# # 구글드라이브 연동
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# 구글 드라이브 파일 확인
!dir 'C:/Users/s/Desktop/와디즈 타이틀 평가(성공확률)/'

# 반복되는 드라이브 경로 변수화
drive_path = '/C:/Users/s/Desktop/와디즈 타이틀 평가(성공확률)/'

스위치가 틀립니다 - "Users".


### 사전점검


In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.__version__, keras.__version__

In [ ]:
# 분산전략 정의
mirrored_strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print('장치의 수:', mirrored_strategy.num_replicas_in_sync)

In [ ]:
# with mirrored_strategy.scope():
#  model = ?

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation, Dropout
from tensorflow.keras.layers import Embedding, LSTM

In [ ]:
import codecs
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys
from string import punctuation
import re

In [ ]:
df = pd.read_csv(drive_path + 'concat_df.csv')
df.info()

In [ ]:
# string으로 type변경
# df = df.astype({'title':'string'})
# print(df.dtypes)

In [ ]:
punctuation

In [ ]:
!pip install konlpy wordcloud

In [ ]:
from konlpy.tag import Kkma, Hannanum, Komoran, Okt

In [ ]:
kkma = Kkma()
han = Hannanum()
kor = Komoran()
okt = Okt()

In [ ]:
#punctutation 제거 패턴
pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))
#숫자 제거 패턴
pattern2 = re.compile(r'[0-9]')

In [ ]:
# 함수화
def cleaning(doc):

  sentence = pattern1.sub(' ',
             pattern2.sub('',str(doc))).strip()

  return sentence

In [ ]:
for i in df['title'][0:30]:
  print(i)
  sentence = pattern1.sub(' ',
             pattern2.sub('',i)).strip()
  print('kkma :',kkma.nouns(sentence))
  print('han :',han.nouns(sentence))
  print('okt :',okt.nouns(sentence))
  print('komoran :',kor.nouns(sentence))

  print('-'*10)

In [ ]:
df['title'] = df['title'].apply(cleaning)
df.head()

In [ ]:
df.to_csv(drive_path+'df_pre.csv')

In [ ]:
df_pre = pd.read_csv(drive_path + 'df_pre.csv')
df_pre.head()

In [ ]:
# fp = codecs.open("./BEXX0003.txt", "r", encoding="utf-16")
# soup = BeautifulSoup(fp, "html.parser")
# body = soup.select_one("body")
# text = body.getText() + " "
print('코퍼스의 길이: ', len(df['title']))

In [ ]:
text =  df['title'].to_string()

In [ ]:
type(text)

In [ ]:
# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자

In [ ]:
chars[0]

In [ ]:
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')

In [ ]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
# 모델 구축하기(LSTM)
print('모델을 구축합니다...')
model = Sequential([
  LSTM(128, input_shape=(maxlen, len(chars)) ),
  Dense(len(chars), activation='softmax')                  
])
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

In [ ]:
# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# # 데이터프레임 리스트로 변환
# df_list = df["title"].tolist()
# type(df_list)

In [ ]:
import ast
# df['title'] = df['title'].apply(ast.literal_eval).str.decode("utf-16")

In [ ]:
start_index = random.randint(0, len(text) - maxlen - 1)
text[start_index: start_index + maxlen]

In [ ]:
# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, epochs=1) # 

    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(df['title']) - maxlen - 1)

    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)

        # 시드를 기반으로 텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            
            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()